In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Reading the data
You might get a `DtypeWarning`; ignore it now, if necessary, deal with it later.

In [ ]:
# Loading the raw dataset
df = pd.read_csv('LendingClub_wi3435TU.csv')

In [ ]:
df.info()

#### Loading the data dictionary
The field descriptions are read in as a Pandas data frame and then extended to a (Pandas) data frame `preview` for a first peek at the data.

In [ ]:
pd.set_option('max_colwidth', 1200) # controls output width; might need adjustment
data_dictionary = pd.read_csv('LCDataDictionary.csv') # Loading in the data dictionary
data_dictionary = data_dictionary.rename(columns={'LoanStatNew': 'name', 'Description': 'description'})
data_dictionary.head()

In [ ]:
# generate preview names + dtypes + first values + descriptions
df_dtypes = pd.DataFrame(df.dtypes,columns=['dtypes'])
df_dtypes = df_dtypes.reset_index()
df_dtypes['name'] = df_dtypes['index']
df_dtypes = df_dtypes[['name','dtypes']]
df_dtypes['first value'] = df.loc[0].values
preview = df_dtypes.merge(data_dictionary, on='name',how='left')

In [ ]:
preview[35:47]

## Features with a lot of missing values
A global investigation into the missing values, using `plt.hist` from Matplotlib.

In [ ]:
# Overall fraction of null values in the data
(df.isnull().sum().sum())/(df.shape[0]*df.shape[1])

In [ ]:
# create overview of distribution of NAs over the variables:
nullfrac= df.isnull().sum()/df.shape[0]
plt.hist(nullfrac,bins=20)
print("Number of fields with more than 10% NAs:", sum(nullfrac>0.10))
print("Number of fields with less than 1%% NAs:", sum(nullfrac<0.001))

In [ ]:
# Example of how to drop the feature from a list
drop_list = ['member_id']
df.drop(drop_list, axis=1, inplace=True)

In [ ]:
df.shape

## Filtering on loan_status

In [ ]:
df['loan_status'].value_counts()

In [ ]:
# value counts, this time fancy in a Pandas data frame
meaning = [
"Loan has been fully paid off.",
"Loan is up to date on current payments.",
"Loan for which there is no longer a reasonable expectation of further payments.",
"Loan hasn't been paid in 31 to 120 days (late on the current payment).",
"The loan is past due but still in the grace period of 15 days.",
"Loan hasn't been paid in 16 to 30 days (late on the current payment).",
"Loan is defaulted on and no payment has been made for more than 121 days."]
status, count = df["loan_status"].value_counts().index, df["loan_status"].value_counts().values
loan_statuses_explanation = pd.DataFrame({'Loan Status': status,'Count': count,'Meaning': meaning})[['Loan Status','Count','Meaning']]
loan_statuses_explanation

In [ ]:
# Example of how you might do this. Complete?
df = df[(df["loan_status"] == "Fully Paid") | (df["loan_status"] == "Charged Off")]
mapping_dictionary = {"loan_status":{ "Fully Paid": 1, "Charged Off": 0, "Default": 0}}
df = df.replace(mapping_dictionary)
df.shape

## Preventing leakage

In [ ]:
# collect list of features that are not available at the application date
drop_list1 = []
df.drop(drop_list1, axis=1, inplace=True)

## Dropping features of no/little predictive value

In [ ]:
# INCOMPLETE
drop_list2 = ['id','disbursement_method']
df.drop(drop_list2, axis=1, inplace=True)

## Unbalanced features

In [ ]:
# an example of how to look at some value-counts
for col in df.columns[26:31]:
    print(df[col].value_counts())
    print()

In [ ]:
# Example
drop_list3 = ['initial_list_status']
df.drop(drop_list3, axis=1, inplace=True)

## Highly correlated features

In [ ]:
# Checking correlation and using heatmap to visualise it.
# First select a subset of the columns to keep the size of the correlation matrix low
# Output looks a bit strange, is it OK?
select = list(df.columns[45:65])     
dfsel = df[select]
sns.set(rc={'figure.figsize':(20,20)})
sns.set_style('whitegrid')
# Compute correlations and plot heatmap (exclude non-numeric features)
correlations = dfsel.corr(numeric_only=True)
sns.heatmap(correlations,annot=True)
plt.show()

## Getting rid of the remaining missing values
Upon first running, before (many) features have been dropped, the list below might/will still contain a large number of feature, many with a large number of missing values. This might indicate you have not cleaned enough in the earlier stages.

In [ ]:
df.shape

In [ ]:
# Create dataframe of null-counts and look at the numbers
# This only makes sense when you have done most of the other cleaning
NAcount= df.isnull().sum()
hasNAs = NAcount[NAcount>0]
print("There are", len(hasNAs), "features that have missing values.")
b = pd.DataFrame(hasNAs,columns=['Number of null values'])
b.sort_values(by=['Number of null values'],ascending=False)

In [ ]:
df.shape # check dimensions

## Dates
There may or may not be any date features left. Let's check the categorical features still present.

In [ ]:
print("Data types and their frequency\n{}".format(df.dtypes.value_counts()))

In [ ]:
# generate list of remaining categorical features
# obcols = list(df.columns[df.dtypes == 'object'])
# obcols

In [ ]:
#for col in obcols:
#    print(preview[preview.name == col][:])

In [ ]:
# this is crummy code ... sorry
pd.set_option('max_colwidth', 20)
object_columns_df = df.select_dtypes(include=['object'])
print(object_columns_df.iloc[0])